# 6. Defining a custom Azure ML Environment  

An **Environment** in Azure ML captures the exact runtime (OS layer, Python packages, system libraries) that your training code needs.  
Instead of starting from a pre-built base image, we point Azure ML to a **local Docker context** (`../docker/`) so it can:

1. Build the Dockerfile into an image inside your Azure Container Registry.  
2. Register that image as a *versioned* Environment object (`<environment_name>:<version>`).  
3. Reuse the same hash-locked image across jobs, ensuring reproducible results.

The helper function below is idempotent: if an environment labelled **latest** already exists, it skips the build; otherwise, it kicks off the build & push.


In [1]:
import yaml

from azure.identity import DefaultAzureCredential

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, BuildContext

## 1) Load settings from config.yaml

In [2]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [3]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

environment_name = config["azure"]["environment_name"]

## 2) Authenticate and connect to the workspace

In [4]:
# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

In [5]:
# Update Azure ML Client
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)

## 3) Create or verify Environment

In [6]:
def create_and_verify_environment(ml_client, environment_name, docker_file_path):
    """
    Creates a new Azure ML environment from a Docker context if it does not already exist.

    This function first checks if an environment with the specified name and label "latest" exists.
    If it does, it prints a message and returns without creating a new environment.
    Otherwise, it builds a new environment using the provided Docker context.

    Parameters:
        ml_client (MLClient): The Azure ML client used to manage environments.
        environment_name (str): The name of the environment.
        docker_file_path (str): The path to the Docker context directory containing the Dockerfile.
            This should be the directory containing the Dockerfile, not the Dockerfile itself.

    Returns:
        None
    """
    try:
        existing_environment = ml_client.environments.get(
            name=environment_name,
            label="latest"
        )
        print(f"Environment '{environment_name}' already exists. No need to recreate.")
        return
    except Exception as e:
        print(f"Environment '{environment_name}' not found. Creating a new one.")

    build_context = BuildContext(
        path=docker_file_path
    )

    env_docker_context = Environment(
        build=build_context,
        name=environment_name,
        description="Environment created from a Docker context.",
        tags={
            "project": "YOLO 11",}
        )

    try:
        created_env = ml_client.environments.create_or_update(env_docker_context)
        print(f"Environment '{created_env.name}' created.")
    except Exception as e:
        print(f"Failed to create or update environment: {e}")


In [ ]:
docker_file_path = "../docker"
create_and_verify_environment(ml_client, environment_name, docker_file_path)

### Tips & next steps

* **Updating the image** – Any change to the Dockerfile automatically triggers a new environment *version*.  
  Keep the same `environment_name` and Azure ML will append an incremental version id (`1`, `2`, …).  
* **Referencing in jobs** – In your training pipeline YAML or SDK call, reference the environment as  
  `azureml:${{AZURE_ML_ENVIRONMENT}}@latest` or pin a specific version for full reproducibility.  
* **Build time** – The first build pushes the image layers to your workspace’s ACR; subsequent builds are much faster due to layer caching.  
* **Docker context size** – Avoid placing large datasets in `../docker/`; only the files needed to build the image should live there.
